In [11]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

In [16]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')



batch_size = 50
# Hyper parameters
num_epochs = 4
num_classes = 10

In [13]:
train_dataset = torchvision.datasets.MNIST(root='data',
                                           train=True, 
                                           transform=transforms.ToTensor())

test_dataset = torchvision.datasets.MNIST(root='data',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [24]:
resnet50 = models.resnet50()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=.001)

resnet50 = resnet50.to(device)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        new_images = np.zeros((50,3,28,28))
        new_images[:,0:] = images 
        
        new_images = torch.FloatTensor(new_images).to(device)
        labels = labels.to(device)
        
        
        
        # Forward pass
        outputs = resnet50(new_images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/4], Step [100/1200], Loss: 0.3148
Epoch [1/4], Step [200/1200], Loss: 0.2144
Epoch [1/4], Step [300/1200], Loss: 0.1339
Epoch [1/4], Step [400/1200], Loss: 0.3035
Epoch [1/4], Step [500/1200], Loss: 0.1997
Epoch [1/4], Step [600/1200], Loss: 0.1801
Epoch [1/4], Step [700/1200], Loss: 0.3041
Epoch [1/4], Step [800/1200], Loss: 0.4560
Epoch [1/4], Step [900/1200], Loss: 0.4571
Epoch [1/4], Step [1000/1200], Loss: 0.1497
Epoch [1/4], Step [1100/1200], Loss: 0.1456
Epoch [1/4], Step [1200/1200], Loss: 0.2124
Epoch [2/4], Step [100/1200], Loss: 0.0563
Epoch [2/4], Step [200/1200], Loss: 0.1345
Epoch [2/4], Step [300/1200], Loss: 0.2947
Epoch [2/4], Step [400/1200], Loss: 0.1125
Epoch [2/4], Step [500/1200], Loss: 0.0056
Epoch [2/4], Step [600/1200], Loss: 0.0110
Epoch [2/4], Step [700/1200], Loss: 0.0572
Epoch [2/4], Step [800/1200], Loss: 0.1451
Epoch [2/4], Step [900/1200], Loss: 0.1219
Epoch [2/4], Step [1000/1200], Loss: 0.1343
Epoch [2/4], Step [1100/1200], Loss: 0.1888
Epoch 

In [26]:
# Test the model
resnet50.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        new_images = np.zeros((50,3,28,28))
        new_images[:,0:] = images 
        
        new_images = torch.FloatTensor(new_images).to(device)
        labels = labels.to(device)
        
        outputs = resnet50(new_images.squeeze())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.49 %
